In [157]:
import numpy as np
import pandas as pd
import jsonlines

import tensorflow as tf
tf.config.run_functions_eagerly(True)

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Flatten, Concatenate
from tensorflow.keras.losses import binary_crossentropy, categorical_crossentropy
from tensorflow.keras.preprocessing.text import Tokenizer
from keras import ops
from tensorflow import keras
from tensorflow.keras.saving import register_keras_serializable

from sklearn.preprocessing import label_binarize

import nltk
nltk.download("stopwords")
import string
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/cpsc477_sh2482/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
file = "data/aspect-level-certainty.jsonl"

annotate_science = []

with jsonlines.open(file) as f:
    for line in f.iter():
        annotate_science.append(line)

In [5]:
finance = pd.read_csv("data/10K_sentiment_list.csv")
uncertain_finance = finance[finance["Uncertainty"] != 0]["Word"].values

In [6]:
# Restructure annotated data from scientific set
label_codes = {"NotPresent": 0, "Certain": 1, "Uncertain": 2}
dims = {"Number": None, "Extent": None, "Probability": None, "Condition": None, "Suggestion": None, "Framing": None}
real_text = [x["finding"] for x in annotate_science]
real_labels = [[label_codes[x["aspect-level-certainty"][dim]] for dim in dims] for x in annotate_science]

In [7]:
latent_dim = 100
vocab_size = 10000
embedding_dim = 128
num_label_dims = 6
one_hot_len = 3
label_values = [0,1,2]
max_len = 64
lstm_units = 64

In [8]:
def preprocess(text):
    """
    Preprocesses text string by removing capitalization, punctuation, and stop words.

    Args:
        text: The text string to be preprocessed.

    Returns:
        The preprocessed text string.
    """

    text = text.lower()
    text = "".join(c for c in text if c not in string.punctuation)
    stop_words = stopwords.words("english")
    text = " ".join([word for word in text.split() if word not in stop_words])
    return text

In [56]:
def tokenize(text_main, text_finance, max_len):
    """
    Tokenizes preprocessed text string.

    Args:
        text: The preprocessed text string.
        max_len: The maximum length for tokenized sequences

    Returns:
        A list of integers representing the tokenized text sequence.
    """
    all_text = text_main + text_finance
    
    tokenizer = Tokenizer(num_words=vocab_size)
    tokenizer.fit_on_texts(all_text)
    vocab = tokenizer.word_index
    
    main_sequences = [tokenizer.texts_to_sequences([line])[0] for line in text_main] 
    for i in range(len(main_sequences)):
        if len(main_sequences[i]) > max_len:
            main_sequences[i] = main_sequences[i][:max_len]
        else:
            main_sequences[i] = tf.keras.utils.pad_sequences([main_sequences[i]], max_len, padding="post")[0]

    finance_tokens = [tokenizer.texts_to_sequences([word])[0] for word in text_finance]
    
    return main_sequences, finance_tokens, vocab

In [158]:
domain_weight = 0.2

@register_keras_serializable(name="generator_loss")
def generator_loss(y_true, y_pred):
    """
    Calculates a custom loss function.

    Args:
        y_true: The ground truth labels.
        y_pred: The predicted labels.
        domain_weight: The weight for finance specific uncertainty loss.

    Returns:
        The combined loss value.
    """
    text_loss = categorical_crossentropy(y_true, y_pred)

    # Make sure to use same embeddings as generator model
    embedding_layer = generator.get_layer('embedding') 
    word_embedding_matrix = embedding_layer.get_weights()[0]

    finance_word_embeddings = tf.nn.embedding_lookup(word_embedding_matrix, finance_token)
    uncertainty_scores = tf.reduce_mean(finance_word_embeddings, axis=1)
    uncertainty_loss = tf.reduce_mean(tf.abs(uncertainty_scores - y_true[:, 1]))
    return text_loss + uncertainty_loss * domain_weight

In [159]:
@register_keras_serializable(name="discriminator_loss")
def discriminator_loss(real_output, fake_output):
    """
    Calculates loss function to distinguish between real and fake text.

    Args:
        real_output: The real text.
        fake_output: The generated tex.

    Returns:
        The loss value.
    """
    real_loss = binary_crossentropy(tf.ones_like(real_output), real_output)
    fake_loss = binary_crossentropy(tf.zeros_like(fake_output), fake_output)
    label_loss = binary_crossentropy(real_output, fake_output)
    return real_loss + fake_loss + label_loss

### Build the Model

In [12]:
def build_generator():
    noise_input = Input(shape=(latent_dim,))
    label_input = Input(shape=(one_hot_len,))
    combined = Concatenate()([noise_input, label_input])
    embedded = Embedding(vocab_size, embedding_dim, name="embedding")(combined)
    lstm1 = LSTM(256, return_sequences=True)(embedded)
    lstm2 = LSTM(128)(lstm1)
    output = Dense(max_len, activation='softmax')(lstm2)
    model = Model(inputs=[noise_input, label_input], outputs=output)
    model.compile(loss = generator_loss, optimizer="adam")
    return model

In [13]:
def build_discriminator():
    text_input = Input(shape=(None,))
    label_input = Input(shape=(one_hot_len,))
    combined = Concatenate()([text_input, label_input])
    embedded = Embedding(vocab_size, embedding_dim)(combined)
    lstm1 = LSTM(256, return_sequences=True)(embedded)
    lstm2 = LSTM(128)(lstm1)
    sentiment_output = Dense(1, activation='sigmoid')(lstm2)
    model = Model(inputs=[text_input, label_input], outputs=sentiment_output)
    model.compile(loss = discriminator_loss, optimizer="adam")
    return model

In [14]:
def build_combined():
    noise_input = Input(shape=(latent_dim,))
    sentiment_input = Input(shape=(one_hot_len,))
    generated_text = generator([noise_input, sentiment_input])
    validity = discriminator([generated_text, sentiment_input])
    model = Model(inputs=[noise_input, sentiment_input], outputs=validity)
    model.compile(loss="categorical_crossentropy", optimizer="adam")
    return model

In [58]:
real_text_process = [preprocess(x) for x in real_text]
finance_text_process = [preprocess(x) for x in uncertain_finance]

real_text_token, finance_token, vocab = tokenize(real_text_process, finance_text_process, max_len)

In [59]:
one_hot_labels = [label_binarize(x, classes=[0,1,2]) for x in real_labels]

dims["Number"] = [x[0] for x in one_hot_labels]
dims["Extent"] = [x[1] for x in one_hot_labels]
dims["Probability"] = [x[2] for x in one_hot_labels]
dims["Condition"] = [x[3] for x in one_hot_labels]
dims["Suggestion"] = [x[4] for x in one_hot_labels]
dims["Framing"] = [x[5] for x in one_hot_labels]

In [61]:
num_epochs = 1
batch_size = 64

num_batches = len(real_text_token) // batch_size

real = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

all_d_loss = [[[0 for x in range(num_batches)] for y in range(num_epochs)] for z in range(num_label_dims)]
all_gen_loss = [[[0 for x in range(num_batches)] for y in range(num_epochs)] for z in range(num_label_dims)]

key_idx = 0
#for key in dims.keys():
for key in ["Number"]:
    print(f"Dimension: {key}")
    labels = dims[key]
    key_idx += 1

    generator = build_generator()
    discriminator = build_discriminator()
    combined = build_combined()
    
    for epoch in range(num_epochs):
        print(f"Epoch: {epoch+1}/{num_epochs}")
        for i in range(num_batches):
        
            start = i * batch_size
            end = (i + 1) * batch_size
            
            real_data = tf.convert_to_tensor(real_text_token[start:end])
            real_data_labels = tf.convert_to_tensor(labels[start:end])
            
            # Train Discriminator
            noise_data = tf.convert_to_tensor(np.random.normal(size=(batch_size, latent_dim)))
            generate_labels = tf.convert_to_tensor(np.random.randint(0, 2, size=(batch_size, one_hot_len)))
    
            generated_data = tf.convert_to_tensor(generator.predict([noise_data, generate_labels]))
            real_data = Flatten()(real_data)
            
            real_loss = discriminator.train_on_batch([real_data, generate_labels], real)
            fake_loss = discriminator.train_on_batch([generated_data, generate_labels], fake)
            d_loss = 0.5 * np.add(real_loss, fake_loss)
    
            all_d_loss[key_idx][epoch][i] = d_loss
            
            # Train Generator        
            gen_loss = combined.train_on_batch([noise_data, real_data_labels], real)
    
            all_gen_loss[key_idx][epoch][i] = gen_loss[0]

            print(f"Batch: {i+1}/{num_batches}")
    

    f1 = f"discriminator_{key}.keras" 
    discriminator.save(f1) 

    f2 = f"generator_{key}.keras"  
    generator.save(f2)

    f3 = f"combined_{key}.keras"  
    combined.save(f3)

Dimension: Number
Epoch: 1/1


/home/cpsc477_sh2482/.conda/envs/myenv/lib/python3.12/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 742ms/step


/home/cpsc477_sh2482/.conda/envs/myenv/lib/python3.12/site-packages/keras/src/losses/losses.py:22: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(64, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)
/home/cpsc477_sh2482/.conda/envs/myenv/lib/python3.12/site-packages/keras/src/optimizers/base_optimizer.py:664: UserWarning: Gradients do not exist for variables ['embeddings', 'kernel', 'recurrent_kernel', 'bias', 'kernel', 'recurrent_kernel', 'bias', 'kernel', 'bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


Batch: 1/27
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 755ms/step
Batch: 2/27
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 743ms/step
Batch: 3/27
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 757ms/step
Batch: 4/27
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 750ms/step
Batch: 5/27
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 752ms/step
Batch: 6/27
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 764ms/step
Batch: 7/27
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 752ms/step
Batch: 8/27
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 759ms/step
Batch: 9/27
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 747ms/step
Batch: 10/27
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 776ms/step
Batch: 11/27
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 763ms/step
Batch: 12/27
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 754ms/step
Batch: 13/27
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 759ms/step
Batch: 14/27
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 777ms/step
Batch: 15/27
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 757ms/step
Batch: 16/27
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 765ms/step
Batch: 17/27
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 759ms/step
Batch: 18/27
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 759ms/step
Batch: 19/27
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 747ms/step
Batch: 20/27
2/2 ━━━━